<a href="https://colab.research.google.com/github/Jiipengentobat/DIF62130_B_25_2311532004/blob/main/praktikum3/NeiveBayesModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(dataset_dict)

df.head()


,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes


In [68]:
df = pd.get_dummies(df, columns=['Outlook'], prefix ='', prefix_sep='', dtype=int)

df['Wind'] = df ['Wind'].astype(int)
df['Play'] = (df ['Play'] == 'Yes').astype(int)

df.head()

,Temperature,Humidity,Wind,Play,overcast,rainy,sunny
0,85.0,85.0,0,0,0,0,1
1,80.0,90.0,1,0,0,0,1
2,83.0,78.0,0,1,1,0,0
3,70.0,96.0,0,1,0,1,0
4,68.0,80.0,0,1,0,1,0


In [69]:
df ['Temperature'] = pd.cut(df['Temperature'], bins=[0,80,100], labels=['warm', 'hot'])
df ['Humidity'] = pd.cut(df['Humidity'], bins=[0,75,100], labels=['dry', 'humid'])

one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']], drop_first=True, dtype=int)
df = df.drop(['Temperature', 'Humidity'], axis=1)
df = pd.concat([ one_hot_columns, df], axis=1)

df.head()

,Temperature_hot,Humidity_humid,Wind,Play,overcast,rainy,sunny
0,1,1,0,0,0,0,1
1,0,1,1,0,0,0,1
2,1,1,0,1,1,0,0
3,0,1,0,1,0,1,0
4,0,1,0,1,0,1,0


In [78]:
x, y = df.drop(columns='Play'), df['Play']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle= False)

In [79]:
y_train.head()

,Play
0,0
1,0
2,1
3,1
4,1


### MAIN MECHANISM

* hitung probabilitas setiap kelas dalam data pelatihan
* untuk etiap fitur dan kelas, hitung probabiliras fitur tersebut bernilai 1 dan 0 berdasarkan kelasnya
* untuk data baru: untuk setiap kelas, kalikan probabilitasnya dengan probabilitas setiap nilai fitur (0 / 1) untuk kelas tersebut
* prediksi kelas dengan probabilitas hasi tertinggi

### TRAINING STEPS
proses pelatihan untuk Bernoulli Naive Bayes melibatkan perhitungan probabilitas dari data pelatihan:
1. class probability calculation : untuk setiap kelas, hitung probalitasnya:(jumlah kejadian di kelas dikelas ini)/ total kejadian

In [94]:
# prompt: hitung rasio

def cal_target_prob(attr):
    total_counts = attr.value_counts().sum()
    prob_series = attr.value_counts().apply(lambda x: Fraction(x, total_counts).limit_denominator())
    return prob_series

print(cal_target_prob(y_train))


Play
1    11/19
0     8/19
Name: count, dtype: object


In [95]:
# prompt: from df, count 1 and 0 and total value and divide it

import pandas as pd

# Assuming 'df' is your DataFrame as defined in the previous code

# Count occurrences of 1 and 0 in the 'Play' column
play_counts = df['Play'].value_counts()
count_1 = play_counts.get(1, 0)  # Use get() to handle cases where 1 might not exist
count_0 = play_counts.get(0, 0)  # Use get() to handle cases where 0 might not exist

# Calculate total count
total_count = len(df['Play'])

# Calculate the ratios (avoid division by zero)
if total_count > 0 :
  ratio_1 = count_1 / total_count
  ratio_0 = count_0 / total_count
else:
  ratio_1 = 0
  ratio_0 = 0


print(f"Count of 1: {count_1}")
print(f"Count of 0: {count_0}")
print(f"Total Count: {total_count}")
print(f"Ratio of 1: {ratio_1}")
print(f"Ratio of 0: {ratio_0}")


Count of 1: 18
Count of 0: 10
Total Count: 28
Ratio of 1: 0.6428571428571429
Ratio of 0: 0.35714285714285715


### 2. feature probablity calculation: untuk setiap fitur dan setiap kelas, hitunglah

* jumlah kejadian dimana fitur adalah 0 di kelas ini / jumlah kejadian kelas ini
* jumlah kejadian dimana fitur adakah 1 di kelas ini / jumlah kejadian kelas ini

In [99]:
from fractions import Fraction

def feature_probabilities(df, target_column):


    feature_probs = {}
    classes = df[target_column].unique()

    for feature in df.columns:
        if feature != target_column:
            feature_probs[feature] = {}
            for cls in classes:
                feature_probs[feature][cls] = {}
                # Filter data for the current class
                class_data = df[df[target_column] == cls]

                # Count occurrences of feature values 0 and 1 in this class
                count_0 = (class_data[feature] == 0).sum()
                count_1 = (class_data[feature] == 1).sum()
                total_count_in_class = len(class_data)

                # Calculate probabilities, handling cases where counts are zero
                if total_count_in_class > 0 :
                    feature_probs[feature][cls]['0'] = count_0 / total_count_in_class
                    feature_probs[feature][cls]['1'] = count_1 / total_count_in_class
                else:
                    feature_probs[feature][cls]['0'] = 0
                    feature_probs[feature][cls]['1'] = 0

    return feature_probs

# Example usage:
feature_probabilities_result = feature_probabilities(df, 'Play')
feature_probabilities_result


{'Temperature_hot': {0: {'0': 0.7, '1': 0.3},
  1: {'0': 0.7222222222222222, '1': 0.2777777777777778}},
 'Humidity_humid': {0: {'0': 0.3, '1': 0.7},
  1: {'0': 0.3888888888888889, '1': 0.6111111111111112}},
 'Wind': {0: {'0': 0.4, '1': 0.6},
  1: {'0': 0.6666666666666666, '1': 0.3333333333333333}},
 'overcast': {0: {'0': 1.0, '1': 0.0}, 1: {'0': 0.5, '1': 0.5}},
 'rainy': {0: {'0': 0.6, '1': 0.4},
  1: {'0': 0.7222222222222222, '1': 0.2777777777777778}},
 'sunny': {0: {'0': 0.4, '1': 0.6},
  1: {'0': 0.7777777777777778, '1': 0.2222222222222222}}}

In [100]:
nb_clf = BernoulliNB()
nb_clf.fit(x_train, y_train)

BernoulliNB()

In [101]:
y_pred = nb_clf.predict(x_test)

In [102]:
print(f"accuracy: {accuracy_score(y_test,y_pred)}")

accuracy: 0.5555555555555556
